#### Загрузка библиотек и необходимых модулей

In [43]:
!pwd
import os
print(os.listdir("data/"))

/resources/Portfolio/Marketing Analytics
['salesDataMon2To4.csv', 'churn_data.csv', '.ipynb_checkpoints', 'online-Copy1.csv', 'online12M.csv', 'online.csv', 'salesData.csv']


In [44]:
import pandas as pd
import numpy as np
import datetime as dt

#### Загрузка и первичная обработка данных

In [45]:
online = pd.read_csv("data/online12M.csv")

In [46]:
online.info()
online.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68176 entries, 0 to 68175
Data columns (total 9 columns):
Unnamed: 0     68176 non-null int64
InvoiceNo      68176 non-null int64
StockCode      68176 non-null object
Description    68176 non-null object
Quantity       68176 non-null int64
InvoiceDate    68176 non-null object
UnitPrice      68176 non-null float64
CustomerID     68176 non-null int64
Country        68176 non-null object
dtypes: float64(1), int64(4), object(4)
memory usage: 4.7+ MB


,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,416792,572558,22745,POPPY'S PLAYHOUSE BEDROOM,6,2011-10-25,2.10,14286,United Kingdom
1,482904,577485,23196,VINTAGE LEAF MAGNETIC NOTEPAD,1,2011-11-20,1.45,16360,United Kingdom
2,263743,560034,23299,FOOD COVER WITH BEADS SET 2,6,2011-07-14,3.75,13933,United Kingdom
3,495549,578307,72349B,SET/6 PURPLE BUTTERFLY T-LIGHTS,1,2011-11-23,2.10,17290,United Kingdom
4,204384,554656,21756,BATH BUILDING BLOCK WORD,3,2011-05-25,5.95,17663,United Kingdom


In [47]:
online.InvoiceDate = pd.to_datetime(online.InvoiceDate) 
online.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68176 entries, 0 to 68175
Data columns (total 9 columns):
Unnamed: 0     68176 non-null int64
InvoiceNo      68176 non-null int64
StockCode      68176 non-null object
Description    68176 non-null object
Quantity       68176 non-null int64
InvoiceDate    68176 non-null datetime64[ns]
UnitPrice      68176 non-null float64
CustomerID     68176 non-null int64
Country        68176 non-null object
dtypes: datetime64[ns](1), float64(1), int64(4), object(3)
memory usage: 4.7+ MB


## RFM-анализ
RFM-анализ — метод анализа, позволяющий сегментировать клиентов по частоте и сумме покупок и выявлять тех клиентов, которые приносят больше денег.

Аббревиатура RFM расшифровывается:

* __Recency (давность)__ — количество дней с момента последней транзакции, чем меньше времени прошло с момента последней активности клиента, тем больше вероятность, что он повторит действие
* __Frequency (частота)__ — количество транзакции в последнии 12 месяцев, чем больше каких-либо действий совершит клиент, тем больше вероятность того, что он его повторит в будущем
* __Monetary (деньги)__ — общая сумма покупок за последние 12 месяцев, тем больше вероятность того, что он сделает заказ

RFM-анализ может быть проведен с использованием нескольких подходов, в данном случае RFM-анализ будет проведен с использованием процентилей.

__Основные шаги при расчете и использовании процентилей в RFM-анализе:__
* 1 шаг Сортировка пользователей с использованием одной из метрик
* 2 шаг Разбивка пользователей на предварительно определенное количество групп примерно одинакового размера
* 3 шаг Присваивание лейбла каждой из групп

## Подготовка и расчет RFM метрик

#### Предварительная подготовка данных 

In [48]:
#расчет нового параметра TotalSum
online['TotalSum'] = online.Quantity * online.UnitPrice

предварительная проверка за какой период у нас есть данные

In [49]:
print('Min:{}; Max:{}'.format(min(online.InvoiceDate), max(online.InvoiceDate)))

Min:2010-12-10 00:00:00; Max:2011-12-09 00:00:00


далее определяем текущий момент времени относительно которого мы будем делать анализ, в данном случае текущий момент будет равен: день последней транзакции + 1 день

In [50]:
snapshot_date = max(online.InvoiceDate) + dt.timedelta(days=1)

#### Расчет RFM метрик

In [51]:
datamart = online.groupby(['CustomerID']).agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'count',
    'TotalSum': 'sum'})

datamart.rename(columns = { 'InvoiceDate': 'Recency',
                            'InvoiceNo': 'Frequency',
                            'TotalSum': 'MonetaryValue'}, inplace=True)
datamart.head()

,Recency,Frequency,MonetaryValue
CustomerID,,,
12747,3,25,948.70
12748,1,888,7046.16
12749,4,37,813.45
12820,4,17,268.02
12822,71,9,146.15


In [52]:
print('Среднее количество транзакций за последние 12 месяцев:{}'.format(np.mean(datamart.Frequency)))

Среднее количество транзакций за последние 12 месяцев:18.71424650013725


## Разбивка клиентов на группы на основе RFM метрик

#### в данном случае найдем квартили (quartile) для каждой метрики Давность, Частота и Деньги

In [53]:
r_labels = range(4, 0, -1)
f_labels = range(1,5)
m_labels = range(1,5)

#найдем квартили (quartile) для каждой метрики
r_quartiles = pd.qcut(datamart['Recency'], 4, labels = r_labels)
f_quartiles = pd.qcut(datamart['Frequency'], 4, labels = f_labels)
m_quartiles = pd.qcut(datamart['MonetaryValue'], 4, labels = m_labels)

datamart = datamart.assign(R = r_quartiles.values)
datamart = datamart.assign(F = f_quartiles.values)
datamart = datamart.assign(M = m_quartiles.values)

In [54]:
datamart.head(3)

,Recency,Frequency,MonetaryValue,R,F,M
CustomerID,,,,,,
12747,3,25,948.70,4,4,4
12748,1,888,7046.16,4,4,4
12749,4,37,813.45,4,4,4


для проведения последующего анализа, нам необходимо ввести две дополнительные переменные __RFM_Segment__ и __RFM_Score__ на основе квартилей:
* __RFM_Segment:__ название сегмента
* __RFM_Score:__ общая сумма найденных квартилей для каждого покупателя

In [55]:
def join_rfm(x): return str(x['R']) + str(x['F']) + str(x['M'])

datamart['RFM_Segment'] = datamart.apply(join_rfm, axis=1)
datamart['RFM_Score'] = datamart[['R','F','M']].sum(axis=1)

In [56]:
datamart.head(3)

,Recency,Frequency,MonetaryValue,R,F,M,RFM_Segment,RFM_Score
CustomerID,,,,,,,,
12747,3,25,948.70,4,4,4,444,12.0
12748,1,888,7046.16,4,4,4,444,12.0
12749,4,37,813.45,4,4,4,444,12.0


## Анализ клиентов на основе полученных данных

#### кол-во покупателей в каждой группе в зависимости от значения переменной RFM_Segment

In [57]:
datamart.groupby('RFM_Segment').size().sort_values(ascending=False)[:10]

RFM_Segment
444    372
111    345
211    169
344    156
233    129
222    128
333    120
122    117
311    114
433    113
dtype: int64

в данном случае большинство клиентов принадлежит 444 группе, наименьшее 433 группе.

также мы можем провести небольшой статистический анализ показателей каждой группы по отдельности, используя для этого значение RFM_Score, и увидеть каким образом варьируются значения Давности, Частоты и Общей суммы покупок, в зависимости от группы к которой принадлежит клиент.

In [58]:
datamart[datamart.RFM_Score == 9 ].mean()

Recency           46.765217
Frequency         16.675362
MonetaryValue    330.269713
R                  2.924638
F                  3.005797
M                  3.069565
RFM_Segment             inf
RFM_Score          9.000000
dtype: float64

#### Разделение клиентов на сегменты в зависимости от значения переменной RFM_Score
В данном случае все покупатели будут поделены на три условных сегмента ('Top', 'Middle' и 'Low') в зависимости от значения переменной RFM_Score

In [59]:
# функция, которая определяет к какой группе принадлежит клиент на основе значения RFM_Score
def rfm_level(df):
    if df['RFM_Score'] >= 10:
        return 'Top'
    elif ((df['RFM_Score'] >= 6) and (df['RFM_Score'] < 10)):
        return 'Middle'
    else:
        return 'Low'

datamart['RFM_Level'] = datamart.apply(rfm_level, axis='columns')

datamart.head(5)

,Recency,Frequency,MonetaryValue,R,F,M,RFM_Segment,RFM_Score,RFM_Level
CustomerID,,,,,,,,,
12747,3,25,948.70,4,4,4,444,12.0,Top
12748,1,888,7046.16,4,4,4,444,12.0,Top
12749,4,37,813.45,4,4,4,444,12.0,Top
12820,4,17,268.02,4,3,3,433,10.0,Top
12822,71,9,146.15,2,2,3,223,7.0,Middle


## Заключительный статистический анализ основных показателей по каждому сегменту
Ниже произведен расчет среднего значения основных маркетологических показателей в зависимости от сегмента, к которому принадлежит клиент, и сколько покупателей принадлежит к тому или иному сегменту.

In [60]:
# расчет среднего значения для каждого параметра и размер каждого сегмента 
rfm_level_agg = datamart.groupby('RFM_Level').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'MonetaryValue': ['mean', 'count']
}).round(1)

# Print the aggregated dataset
rfm_level_agg

Recency Frequency MonetaryValue      
             mean      mean          mean count
RFM_Level                                      
Low         180.8       3.2          52.7  1075
Middle       73.9      10.7         202.9  1547
Top          20.3      47.1         959.7  1021

## Вывод
В данном кейсе произведен RFM-анализ в результате которого мы произвели расчет основных маркетологических метрик: Recency (давность), Frequency (частота) и Monetary (деньги). Далее мы разбили всех клиентов на три условных сегмента ('Top', 'Middle' и 'Low'), в дальнейшем это позволит выявить наиболее ценных клиентов для компании и проводить различные маркетинговые компании уже с учетом ценности клиента.

<pr>Стоить отметить что в данном случае количество сегментов было выбрано условно. Зачастую на практике при анализе больших баз данных это довольно трудно сделать, поэтому для сегментации рынка применяются различные методы машинного обучения, такие как метод k-средних. Пример с использованием машинного обучения на неразмеченных данных для сегментации рынка приведен в файле Customer Segmentation with Unsupervised ML.ipynb.